### 3. Kommunikation mit dem Netzwerk  - Teil 2: Transaktionen

Basierend auf der Flask Implementierung sollen nun die `FullNode` und die `Wallet` Klasse so erweitert werden, dass sowohl *FullNodes* als auch *Wallets* Transaktionen an andere *FullNodes* weiterreichen (bzw. im Fall von Wallets initial posten) können.   
*Hinweis: Wir habe zwar bisher mit Hilfe von Wallets schon Transaktionen erzeugt und durch einen FullNode verarbeiten lassen, dabei haben wir aber stets direkt auf die Methoden einer `FullNode` Instanz zugegriffen. Das hat zu Testzwecken so funktioniert, ist aber kein realistisches Setting, da eine Wallet in der Realität wohl keine `FullNode` Funktionen aufrufen kann.*  
Wir erweitern die `run_full_node.py` Datei um den folgenden Anker für das Posten von Transaktionen:

```python
# This is the anchor through which Transactions can be posted to the node.
@app.route("/post_transaction", methods=['POST'])
def post_transaction():
    # Parses the Transaction Data and calls the full_node
    # processing the transaction.

    transaction = Transaction.from_odict(create_odict_from_json(request.get_data()))
    response = full_node.add_transaction_to_mempools(transaction)
    return json.dumps(response), 200
```
Im Gegensatz zu den bisher verwendeten Ankern erlauben wir den Zugriff auf diesen Ankern nicht per `get` sondern per `post` Request. Bzgl. der Funktionalität wäre auch hier `get` möglich (die Unterschiede zwischen `get` und `post` Requests sind für unsere Anwendung nicht so wichtig). Um uns jedoch an die allgemeine Konvention zu halten, erlauben wir hier den `post` Request, da der wesentliche Teil der Daten bei diesem Anker an den *FullNode* gesendet (also *gepostet*) werden soll. 

#### Wallet Klasse

Zusätzlich erweitern wir die `Wallet` Klasse um die folgenden Methoden:

```python
    ## New (4_3) 1:
    def create_signed_transaction(self, recipient_pub_key, value):
        # Creates a signed Transaction with UTXOs from the own UTXO Pool.
        # The Wallet has to be connected to a parent FullNode to
        # be able to run this method. Receiver is recipient_pub_key, value is self
        # explaining. Outputs can be max. 2 TOs (one main, one change).

        # Update balance
        if self.parent_node is not None:
            self.calculate_balance()

            # Check if balance is sufficient
            if value > self.balance:
                raise ValueError('Value exceeds balance')
                

            #  Loop through UTXOs up to the point where the value is exceeded
            sum_of_UTXOs = 0
            used_UTXO_ids = []
            for UTXO_id, UTXO in self.UTXOs.items():
                sum_of_UTXOs += UTXO.value
                used_UTXO_ids.append(UTXO.id)
                if sum_of_UTXOs >= value:
                    break

            outputs = OrderedDict()
            recipient_output = TransactionOutput(recipient_pub_key,value)
            outputs[recipient_output.id] = recipient_output

            # If sum doesn't exactly add up, create change transaction to own wallet
            if sum_of_UTXOs > value:
                change_output = TransactionOutput(self.public_key,sum_of_UTXOs-value)
                outputs[change_output.id] = change_output
            gen_transaction = Transaction(self.public_key,used_UTXO_ids,outputs)
            gen_transaction.sign_transaction(self.private_key)
            return(gen_transaction)
        else:
            print('No parent not connected, please connect parent node first.')
            return(None)
            
    ## New (4_3) 2:
    def post_signed_transaction(self, signed_transaction):
        # Posts the signed_transaction to the parent full node.
        
        request_data = json.dumps(signed_transaction.get_full_odict()).encode('utf8')
        response = requests.post('http://' + self.__parent_node  \
        + '/post_transaction', data = request_data)
        return(response)
```

* **## New (4_3) 1**: Die Methode `create_signed_transaction(self, recipient_pub_key, value)` erzeugt eine signierte Transaktion an `recipient_pub_key` mit dem Wert `value`. Hierzu werden die `UTXOs` verwendet, die der `Wallet` gehören. In vielen Fällen ist die Summe der Werte der verwendeten `UTXOs` ungleich (größer) `value`. Dann wird automatisch ein Wechselgeld `TransactionOutput` an die eigene Walletadresse mit dem entsprechenden Differenzbetrag der Transaktion hinzugefügt.
<br/>
<br/>
* **## New (4_3) 2**: Die Methode `post_signed_transaction(self, signed_transaction)` erzeugt den *Post-Request*, der die signierte Transaktion an den *FullNode* übermittelt, und führt diesen Request auch aus.

Um die Implementierung zu testen, wechsle innerhalb Deiner Konsole in dieses Verzeichniss. Starte im Anschluss daran den *FullNode* über die Ausführung der `run_full_node.py` Datei auf dem Port `5001`. Anschließend kannst Du die Implementierung von diesem Notebook aus testen. In der Konsole wird dir ein Zugriff auf eine Methode über den Node auch immer angezeigt (mit der entsprechenden Information, ob es sich um einen `get` oder `post` Request handelt). 

In [ ]:
# Automatically relaoad modules that have changed while
# being loaded.
%load_ext autoreload
# Set equal to 2 to enable auto realod,
# set equal to 0 to disable auto relaod.
%autoreload 2

In [ ]:
# Import stuff
# Import the modules we need
from IPython.core.debugger import set_trace

from FEDCoin import *
from utils import *

In [ ]:
# Testing it:

## Same as 4_2_Teil_1 - start ##
gen_priv_key = '308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042012bbcc17335fa7345f7be65b01a5d6dca706d8f2ba99691f314697f288d678c0a144034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e'
parent_address = '127.0.0.1:5001'
gen_wallet = Wallet(gen_priv_key, parent_address)

# Check the balance (should output 50):
print(gen_wallet.balance)
## Same as 4_2_Teil_1 - end ##

# Creste a new receiver wallet (more or less random, doesn't
# have to be connected)
m_wallet = Wallet()

# Create signed Transaction and post it.
signed_transaction = gen_wallet.create_signed_transaction(m_wallet.public_key, 31)
gen_wallet.post_signed_transaction(signed_transaction)

### FullNode Klasse (und `run_full_node.py`)

#### Registrierung von Nachbarn

Ein *FullNode* ist meistens mit mehreren anderen *FullNodes* im Netzwerk verbunden. Sobald der *FullNode* eine neue Transaktion erhält - entweder von einer *Wallet* oder von einem anderen *FullNode* - versucht er die Transaktion zu den *Mempools* seiner *Blockchains* hinzufügen. Gelingt dies bei der *Primary-Chain*, so leitet der *FullNode* die Transaktion an seine Nachbarn weiter. Damit sich Nachbarn dynamisch untereinander bekannt machen können, muss es für einen *FullNode* möglich sein, sich als Nachbar bei einem anderen *FullNode* vorzustellen. Gleichzeitig muss der *FullNode* auch in der Lage sein, die Vorstellung eines anderen *FullNodes* bei ihm selbst zu verarbeiten. Hierfür führen wir zwei neue Methoden in der `FullNode` Klasse ein:

```python
    ## New (4_2_Teil_2) 3:
    def register_a_neighbor(self, neighbor_address):
        # Registers a neighbor and returns list of other neighbors
        # neglecting the new registered one.
        # This method handles *incoming* neighboring requests.

        # Check if neighbor is registered yet, if not register
        if neighbor_address not in self.neighbors:
            self.neighbors.append(neighbor_address)
            print(str(self.address) + ' added neighbor ' +  str(neighbor_address))
        # 
        return [x for x in self.neighbors if x != neighbor_address]
    
    ## New (4_2_Teil_2) 4:
    def register_as_neighbor(self, parent_node):
        # Registers as neighbor at the parent_node and adds the neighbors
        # of the parent node to the own neighbors list (if not yet part of it).
        # This method posts *outgoing* neighboring requests.
        # Note that we use the 'get' request here since as return we want to get
        # the list of the parent_node's neighbors.

        # Pass the own address
        request_data = {'address' : self.address}
        parent_neighbors_json = requests.get('http://' + parent_node  \
        + '/register_a_neighbor', data = request_data).content
        # Make a list out of it
        parent_neighbors = json.loads(parent_neighbors_json)
        if len(parent_neighbors) > 0:
            # Go through the neighbors and add them if they are not already known.
            # Also process the new neighbor's neighbors (recursively).
            for n in parent_neighbors:
                if n not in self.neighbors:
                    self.neighbors.append(n)
                    print(str(self.address) + ' added neighbor ' + str(n))
                    # Recursion
                    self.register_as_neighbor(n)

```

* **## New (4_2_Teil_2) 3**: Die Methode `register_a_neighbor` wird aufgerufen, wenn sich ein anderer *FullNode* als Nachbar bei diesem *FullNode* registrieren möchte. Ist die Adresse des aufrufenden *FullNodes* - `neighbor_address` - noch nicht registriert, wird sie der `self.neighbors` Liste hinzugefügt. In jedem Fall wird die Liste der Adressen aller bereits registrierten Nachbarn dieses *FullNodes* zurückgegeben. Mit dieser Info kann sich der aufrufende *FullNode*  auch noch bei den Nachbarn registrieren. Um die `register_a_neighbor` Funktion für den aufrufenden *FullNode* zugänglich zu machen, verwenden wir den folgenden Flask Anchor (in `run_full_node.py`):

  ```python
  ## New (4_3) run 1:
  @app.route("/register_a_neighbor", methods=['GET'])
  def register_a_neighbor():
      other_neighbors = full_node.register_a_neighbor(request.form['address'])
      return json.dumps(other_neighbors), 200
  ```
* **## New (4_2_Teil_2) 4**: Die Methode `register_as_neighbor` wird vom registrierenden *FullNode* aufgerufen, um sich als Nachbar bei einem anderen *FullNode* zu registrieren. Innerhalb der Methode wird zuerst der *Get-Request* ausgeführt, über den *register_a_neighbor* beim empfangenden *FullNode* getriggert wird. Außerdem werden auch die Rückgabewerte (die Adressen der Nachbar des empfangenden *FullNodes*) per Rekursion verarbeitet.


Damit ein *FullNode* Teil des bestehenden Netztes werden kann (Ausnahme der allererste *FullNode* des Netzes), fügen wir außerdem die folgenden Zeilen in der `run_full_node.py` Datei hinzu. Dadurch wird die Übergabe von Nachbaradressen bei Aufruf der Datei ermöglicht. Dieses Nachbaradressen sind dann der Einstieg in das Netz. Prinziell genügt die Übergabe der Adresse eines Nachbars, da durch die Nachbarsnachbar Funktion dann die weitere Vernetzung erfolgen kann. Durch die Übergabe von `nargs='*'` sagen wir der `add_argument` Methode, dass auch mehrere Nachbar innerhalb eines Aufrufs übergeben werden können. Diese müssen dann einfach durch Leerzeichen voneinander getrennt werden.

```python
    ## New (4_3) run 2:
    parser.add_argument('-n','--neighbors', nargs='*', help='Neighbor addresses, i.e. 127.0.0.1:5000')
...
    ## New (4_3) run 3:
    neighbors = args.neighbors
...
    ## New (4_3) run 4:
    # Register as neighbor
    if neighbors is not None:
        for n in neighbors:
            # Add the neighbor to the node's own list of neighbors
            full_node.neighbors.append(n)
            # Register at the node as neighbor 
            # (i.e. the other node adds this node to its list of neighbors as well)
            full_node.register_as_neighbor(n)
```
Der Code ist den Implementierungen in diesem Verzeichniss bereits hinzugefügt. 

#### Testing it:
Um die Nachbarregistrierung zu testen, starte drei *FullNodes* in drei separaten Konsolen (Konsolen müssen in dieses Verzeichniss navigieren).  
*Hinweis: Achte darauf, dass die alten FullNodes vorher alle beendet wurden (Konsole schließen oder Str+C drücken). Ansonsten kann es zu Konflikten kommen.*  
Einen auf Port `5000` (*FullNode A*) den zweiten auf Port `5001` (*FullNode B*) und den dritten auf Port `5002` (*FullNode C*). Beim Start der beiden *FullNodes B* und *C* übergebe zusätzlich noch die Adresse von *FullNode A* als Nachbar. D.h. in den drei Konsolen wird jeweils folgender Code (in dieser Reihenfolge) ausgeführt:
* Konsole *FullNode A*:  
  `python run_full_node.py -p 5000`
  
* Konsole *FullNode B*:  
  `python run_full_node.py -p 5001 -n 127.0.0.1:5000`
  
* Konsole *FullNode C*:  
  `python run_full_node.py -p 5002 -n 127.0.0.1:5000`

*Wichtig: In jeder Konsoloe muss das `fed_coin` Environment zu Beginn separat aktiviert werden, ansonsten kann `run_full_node.py`selbstverständlich in der entsprechenden Konsole nicht gestartet werden, weil die benötigten Module fehlen.*  
Durch die im Code hinterlegten `print` Statements, kann überprüft werden, ob alles nach Plan verlaufen ist. Jeder der drei Knoten sollte am Ende mit jedem vernetzt sein. Durch den Outputd der Konsolen sollte die Vernetzung bestätigt werden.

### Weiterleiten von Transaktionen

Damit Transaktionen per *Wallet* bzw. von *FullNode* zu *FullNode* gepostet werden kann, haben wir wir oben den Anker `@app.route("/post_transaction", methods=['POST'])` in `run_full_node.py` hinzugefügt. Einzig die automatische Weiterleitung einer beim *FullNode* eingehenden Transaktion ist bisher noch nicht implementiert. Hierfür wird innerhalb der `full_node.add_transaction_to_mempools()` Methode die `propagate_transaction()` aufgerufen. `propagate_transaction()` ist bisher nur als Dummy implementiert. Wir vervollständigen diese Methode nun so, dass eine Transaktion immer an zwei zufällig ausgewählte Nachbar weitegleitet werden soll (sofern die Transaktion dem Mempool der *Primary-Chain* hinzugefügt wurde. Die zufällige Auswahl der Nachbar erfolgt mit der `get_neigbor_selection()` Methode:

```python

    ## New (4_3) 5:
    def propagate_transaction(self, transaction):
        # Posts the transaction to randomly selected set
        # of neighbors.

        transaction_json = json.dumps(transaction.get_full_odict()).encode('utf8')
        selected_neighbors = self.get_neighbor_selection(2)
        if selected_neighbors:
            for n in selected_neighbors:
                result = requests.post('http://' + n  + '/post_transaction', data = transaction_json)
                

    ## New (4_3) 6:
    def get_neighbor_selection(self, max_number_of_neighbors):
        # Generates and returns a random set of neighbors.
        # Returns None if no neighbor is set.
        
        # Set max number of neighbors to post to here
        num_to_select = min(len(self.neighbors),max_number_of_neighbors)
        if num_to_select>0:
            selected_neighbors = random.sample(self.neighbors, num_to_select)
            return selected_neighbors
        else:
            return None

```


Der Code ist der Implementierung in diesem Verzeichniss bereits hinzugefügt. Teste, ob die Transaktionsweiterleitung funktioniert, indem Du eine Transaktion von einer Genesis Wallet an eine beliebige andere Wallet an das Netzwerk sendest.

In [ ]:
# Testing it:
# Note that we use exactly the same code as above. We only check if after posting the
# transaction to the :5000 Node it is porpagated correclty to the two other nodes in the network.
# To see that we look for the print statements in the console.

gen_priv_key = '308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042012bbcc17335fa7345f7be65b01a5d6dca706d8f2ba99691f314697f288d678c0a144034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e'
parent_address = '127.0.0.1:5000'
gen_wallet = Wallet(gen_priv_key, parent_address)

# Check the balance (should output 50):
print(gen_wallet.balance)
## Same as 4_2_Teil_2 --end##

# Creste a new receiver wallet (more or less random, doesn't
# have to be connected)
m_wallet = Wallet()

# Create signed Transaction and post it.
signed_transaction = gen_wallet.create_signed_transaction(m_wallet.public_key, 31)
gen_wallet.post_signed_transaction(signed_transaction)

# Result:
# Chekcking the print outputs in the console you should see that
# all nodes added the transaction to its refering mempool.